In [1]:
%reload_ext autoreload
%autoreload 2

import os
import pickle
import logging
import multiprocessing
from pathlib import Path

import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp

import arviz as az
import numpyro

from hbmep.config import Config
from hbmep.model.utils import Site as site

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()
numpyro.enable_validation()

logger = logging.getLogger(__name__)


In [2]:
import numpyro.distributions as dist
from hbmep.model import BaseModel


class MixtureModel(BaseModel):
    LINK = "mixture_model"

    def __init__(self, config: Config):
        super(MixtureModel, self).__init__(config=config)
        self.combination_columns = self.features + [self.subject]

    def fn(self, x, a, b, v, L, l, H):
        return (
            L
            + jnp.where(
                jnp.less(x, a),
                0.,
                -l + jnp.true_divide(
                    H + l,
                    jnp.power(
                        1
                        + jnp.multiply(
                            -1
                            + jnp.power(
                                jnp.true_divide(H + l, l),
                                v
                            ),
                            jnp.exp(jnp.multiply(-b, x - a))
                        ),
                        jnp.true_divide(1, v)
                    )
                )
            )
        )

    def _model(self, subject, features, intensity, response_obs=None):
        subject, n_subject = subject
        features, n_features = features
        intensity, n_data = intensity

        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0].reshape(-1,)
        n_feature0 = n_features[0]

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            """ Global Priors """
            a_mean_global_scale = numpyro.sample("a_mean_global_scale", dist.HalfNormal(100))
            a_shape_global_scale = numpyro.sample("a_shape_global_scale", dist.HalfNormal(100))

            b_scale_global_scale = numpyro.sample("b_scale_global_scale", dist.HalfNormal(100))
            v_scale_global_scale = numpyro.sample("v_scale_global_scale", dist.HalfNormal(100))

            L_scale_global_scale = numpyro.sample("L_scale_global_scale", dist.HalfNormal(10))
            l_scale_global_scale = numpyro.sample("l_scale_global_scale", dist.HalfNormal(100))
            H_scale_global_scale = numpyro.sample("H_scale_global_scale", dist.HalfNormal(10))

            g_1_scale_global_scale = numpyro.sample("g_1_scale_global_scale", dist.HalfNormal(100))
            g_2_scale_global_scale = numpyro.sample("g_2_scale_global_scale", dist.HalfNormal(100))

            print("a_mean_global_scale", a_mean_global_scale.shape)

            # p_scale_global_scale = numpyro.sample("p_scale_global_scale", dist.HalfNormal(100))

            with numpyro.plate("n_feature0", n_feature0, dim=-2):
                """ Hyper-priors """
                # a_mean = numpyro.sample("a_mean", dist.HalfNormal(scale=100))
                # a_scale = numpyro.sample("a_scale", dist.HalfNormal(scale=100))

                # a_mean = numpyro.sample("a_mean", dist.HalfNormal(scale=50))
                # a_shape = numpyro.sample("a_shape", dist.HalfNormal(scale=100))

                a_mean_raw = numpyro.sample("a_mean_raw", dist.HalfNormal(scale=1))
                a_mean = numpyro.deterministic("a_mean", jnp.multiply(a_mean_global_scale, a_mean_raw))

                a_shape_raw = numpyro.sample("a_shape_raw", dist.HalfNormal(scale=1))
                a_shape = numpyro.deterministic("a_shape", jnp.multiply(a_shape_global_scale, a_shape_raw))

                b_scale_raw = numpyro.sample("b_scale_raw", dist.HalfNormal(scale=1))
                b_scale = numpyro.deterministic("b_scale", jnp.multiply(b_scale_global_scale, b_scale_raw))

                v_scale_raw = numpyro.sample("v_scale_raw", dist.HalfNormal(scale=1))
                v_scale = numpyro.deterministic("v_scale", jnp.multiply(v_scale_global_scale, v_scale_raw))

                L_scale_raw = numpyro.sample("L_scale_raw", dist.HalfNormal(scale=1))
                L_scale = numpyro.deterministic("L_scale", jnp.multiply(L_scale_global_scale, L_scale_raw))

                l_scale_raw = numpyro.sample("l_scale_raw", dist.HalfNormal(scale=1))
                l_scale = numpyro.deterministic("sigma_l", jnp.multiply(l_scale_global_scale, l_scale_raw))

                H_scale_raw = numpyro.sample("H_scale_raw", dist.HalfNormal(scale=1))
                H_scale = numpyro.deterministic("H_scale", jnp.multiply(H_scale_global_scale, H_scale_raw))

                g_1_scale_raw = numpyro.sample("g_1_scale_raw", dist.HalfNormal(scale=1))
                g_1_scale = numpyro.deterministic("g_1_scale", jnp.multiply(g_1_scale_global_scale, g_1_scale_raw))

                g_2_scale_raw = numpyro.sample("g_2_scale_raw", dist.HalfNormal(scale=1))
                g_2_scale = numpyro.deterministic("g_2_scale", jnp.multiply(g_2_scale_global_scale, g_2_scale_raw))

                # p_scale_raw = numpyro.sample("p_scale_raw", dist.HalfNormal(scale=1))
                # p_scale = numpyro.deterministic("p_scale", p_scale_global_scale * p_scale_raw)
                print("a_mean", a_mean.shape)

                with numpyro.plate(site.n_subject, n_subject, dim=-3):
                    """ Priors """
                    a_raw = numpyro.sample("a_raw", dist.Gamma(concentration=a_shape, rate=1))
                    a = numpyro.deterministic(site.a, jnp.true_divide(jnp.multiply(a_raw, a_mean), a_shape))
                    # a = numpyro.sample(site.a, dist.TruncatedNormal(loc=a_mean, scale=a_scale, low=0))

                    b_raw = numpyro.sample("b_raw", dist.HalfNormal(scale=1))
                    b = numpyro.deterministic(site.b, jnp.multiply(b_scale, b_raw))

                    v_raw = numpyro.sample("v_raw", dist.HalfNormal(scale=1))
                    v = numpyro.deterministic(site.v, jnp.multiply(v_scale, v_raw))

                    L_raw = numpyro.sample("L_raw", dist.HalfNormal(scale=1))
                    L = numpyro.deterministic(site.L, jnp.multiply(L_scale, L_raw))

                    l_raw = numpyro.sample("l_raw", dist.HalfNormal(scale=1))
                    l = numpyro.deterministic("l", jnp.multiply(l_scale, l_raw))

                    H_raw = numpyro.sample("H_raw", dist.HalfNormal(scale=1))
                    H = numpyro.deterministic(site.H, jnp.multiply(H_scale, H_raw))

                    g_1_raw = numpyro.sample("g_1_raw", dist.HalfCauchy(scale=1))
                    g_1 = numpyro.deterministic(site.g_1, jnp.multiply(g_1_scale, g_1_raw))

                    g_2_raw = numpyro.sample("g_2_raw", dist.HalfCauchy(scale=1))
                    g_2 = numpyro.deterministic(site.g_2, jnp.multiply(g_2_scale, g_2_raw))

                    # p_raw = numpyro.sample("p_raw", dist.HalfNormal(scale=1))
                    # p = numpyro.deterministic("p", p_scale * p_raw)
                    print("a", a.shape)


        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate("n_feature0", n_feature0, dim=-2):
                """ Outlier Distribution """
                outlier_shape = numpyro.sample("outlier_shape", dist.HalfNormal(5))
                outlier_rate = numpyro.sample("outlier_rate", dist.HalfNormal(1))
                outlier_prob = numpyro.sample("outlier_prob", dist.Uniform(0., .2))

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate(site.n_data, n_data, dim=-2):
                """ Model """
                mu = numpyro.deterministic(
                    site.mu,
                    self.fn(
                        x=intensity,
                        a=a[subject, feature0],
                        b=b[subject, feature0],
                        v=v[subject, feature0],
                        L=L[subject, feature0],
                        l=l[subject, feature0],
                        H=H[subject, feature0]
                    )
                )
                beta = numpyro.deterministic(
                    site.beta,
                    g_1[subject, feature0] + jnp.true_divide(g_2[subject, feature0], mu)
                )

                q = numpyro.deterministic("q", outlier_prob[feature0])
                bg_shape = numpyro.deterministic("bg_shape", outlier_shape[feature0])
                bg_rate = numpyro.deterministic("bg_rate", outlier_rate[feature0])
                print("q", q.shape, "bg_shape", bg_shape.shape, "bg_rate", bg_rate.shape)
                print("probs", jnp.stack([1 - q, q], axis=-1).shape)
                print("conc", jnp.stack([jnp.multiply(mu, beta), bg_shape], axis=-1).shape)
                print("rate", jnp.stack([beta, bg_rate], axis=-1).shape)

                print("categorical_batch_shape", dist.Categorical(probs=jnp.stack([1 - q, q], axis=-1)).batch_shape)

                """ Mixture """
                Mixture = dist.MixtureSameFamily(
                    mixing_distribution = dist.Categorical(
                        probs=jnp.stack([1 - q, q], axis=-1)
                    ),
                    component_distribution=dist.Gamma(
                        concentration=jnp.stack(
                            [jnp.multiply(mu, beta), bg_shape],
                            axis=-1
                        ),
                        rate=jnp.stack(
                            [beta, bg_rate],
                            axis=-1
                        )
                    )
                )

                """ Observation """
                numpyro.sample(
                    site.obs,
                    Mixture,
                    obs=response_obs
                )



In [3]:
toml_path = "/home/vishu/repos/hbmep-paper/configs/paper/tms/proc-2023-11-13/config.toml"

config = Config(toml_path=toml_path)
config.BUILD_DIR = os.path.join(config.BUILD_DIR, "model-comparison/mixture-model/grade/")
config.RESPONSE = ["PKPK_ADM"]

config.MCMC_PARAMS["num_warmup"] = 5000
config.MCMC_PARAMS["num_samples"] = 1000

model = MixtureModel(config=config)


2023-11-15 14:28:58,907 - hbmep.config - INFO - Verifying configuration ...
2023-11-15 14:28:58,908 - hbmep.config - INFO - Success!


2023-11-15 14:28:59,087 - hbmep.model.baseline - INFO - Initialized base_model


In [4]:
src = "/home/vishu/data/hbmep-processed/human/tms/proc-2023-11-15.csv"
df = pd.read_csv(src)

df[model.features[0]] = df[model.features[0]].replace({
    "Uninjured": "01_Uninjured",
    "SCI": "02_SCI"
})

subset = ["SCA01", "SCS04"]
ind = df[model.subject].isin(subset)
df = df[ind].reset_index(drop=True).copy()

# ind = df[model.intensity] > 0
# df = df[ind].reset_index(drop=True).copy()

df, encoder_dict = model.load(df=df)


2023-11-15 14:28:59,318 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/paper/tms/proc-2023-11-13/model-comparison/mixture-model/grade/
2023-11-15 14:28:59,332 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/paper/tms/proc-2023-11-13/model-comparison/mixture-model/grade/
2023-11-15 14:28:59,334 - hbmep.dataset.core - INFO - Processing data ...
2023-11-15 14:28:59,335 - hbmep.utils.utils - INFO - func:load took: 0.02 sec


In [5]:
mcmc, posterior_samples = model.run_inference(df=df)


2023-11-15 14:28:59,406 - hbmep.model.baseline - INFO - Running inference with base_model ...


a_mean_global_scale (1,)
a_mean (2, 1)
a (2, 2, 1)
q (157, 1) bg_shape (157, 1) bg_rate (157, 1)
probs (157, 1, 2)
conc (157, 1, 2)
rate (157, 1, 2)
categorical_batch_shape (157, 1)
a_mean_global_scale (1,)
a_mean (2, 1)
a (2, 2, 1)
q (157, 1) bg_shape (157, 1) bg_rate (157, 1)
probs (157, 1, 2)
conc (157, 1, 2)
rate (157, 1, 2)
categorical_batch_shape (157, 1)


  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

a_mean_global_scale (1,)
a_mean (2, 1)
a (2, 2, 1)
q (157, 1) bg_shape (157, 1) bg_rate (157, 1)
probs (157, 1, 2)
conc (157, 1, 2)
rate (157, 1, 2)
categorical_batch_shape (157, 1)
a_mean_global_scale (1,)
a_mean (2, 1)
a (2, 2, 1)
q (157, 1) bg_shape (157, 1) bg_rate (157, 1)
probs (157, 1, 2)
conc (157, 1, 2)
rate (157, 1, 2)
categorical_batch_shape (157, 1)


2023-11-15 14:38:02,421 - hbmep.utils.utils - INFO - func:run_inference took: 9 min and 3.02 sec


In [ ]:
prediction_df = model.make_prediction_dataset(df=df)
posterior_predictive = model.predict(df=prediction_df, posterior_samples=posterior_samples)

model.render_recruitment_curves(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples, prediction_df=prediction_df, posterior_predictive=posterior_predictive)
model.render_predictive_check(df=df, encoder_dict=encoder_dict, prediction_df=prediction_df, posterior_predictive=posterior_predictive)


2023-11-14 16:34:06,803 - hbmep.utils.utils - INFO - func:make_prediction_dataset took: 0.01 sec
2023-11-14 16:34:12,439 - hbmep.utils.utils - INFO - func:predict took: 5.64 sec
2023-11-14 16:34:12,439 - hbmep.model.baseline - INFO - Rendering recruitment curves ...
2023-11-14 16:34:15,201 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/paper/tms/proc-2023-11-13/model-comparison/mixture-model/fix/recruitment_curves.pdf
2023-11-14 16:34:15,201 - hbmep.utils.utils - INFO - func:render_recruitment_curves took: 2.76 sec
2023-11-14 16:34:15,202 - hbmep.model.baseline - INFO - Rendering posterior predictive checks ...
2023-11-14 16:34:18,334 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/paper/tms/proc-2023-11-13/model-comparison/mixture-model/fix/posterior_predictive_check.pdf
2023-11-14 16:34:18,334 - hbmep.utils.utils - INFO - func:predictive_checks_renderer took: 3.13 sec
2023-11-14 16:34:18,334 - hbmep.utils.utils - INFO - f

In [ ]:
mcmc.print_summary(prob=.95)



                               mean       std    median      2.5%     97.5%     n_eff     r_hat
             H_raw[0,0,0]      0.78      0.56      0.68      0.03      1.87     42.21      1.11
             H_raw[0,1,0]      0.68      0.21      0.67      0.32      1.08     41.81      1.10
             H_raw[1,0,0]      0.78      0.60      0.63      0.00      1.95     76.68      1.06
             H_raw[1,1,0]      0.69      0.49      0.52      0.15      1.83      8.97      1.50
             H_raw[2,0,0]      0.81      0.59      0.71      0.00      1.95    109.46      1.03
             H_raw[2,1,0]      0.25      0.07      0.24      0.12      0.39     39.35      1.14
             H_raw[3,0,0]      0.83      0.59      0.71      0.00      2.02    128.25      1.01
             H_raw[3,1,0]      0.70      0.56      0.56      0.00      1.82     94.24      1.05
             H_raw[4,0,0]      0.69      0.56      0.56      0.02      1.79     28.88      1.13
             H_raw[4,1,0]      0.76    

In [11]:
import scipy.stats as stats

a = posterior_samples["a"]
a = np.array(a)
a_map = a.mean(axis=0)[..., 0]

stats.ttest_ind(a=a_map[:10, 1], b=a_map[10:, 0], alternative="greater")

TtestResult(statistic=0.6538370230861886, pvalue=0.26190594190573013, df=14.0)

In [8]:
numpyro_data = az.from_numpyro(mcmc)

""" Model evaluation """
logger.info("Evaluating model ...")

score = az.loo(numpyro_data)
logger.info(f"ELPD LOO (Log): {score.elpd_loo:.2f}")

score = az.waic(numpyro_data)
logger.info(f"ELPD WAIC (Log): {score.elpd_waic:.2f}")


2023-11-14 16:36:02,514 - __main__ - INFO - Evaluating model ...
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:1037: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:803: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
2023-11-14 16:36:04,824 - __main__ - INFO - ELPD LOO (Log): 2839.17
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:1645: UserWarning: For one or more samples the posterior variance of the log predictive densities

In [ ]:
import pickle

dest = os.path.join(model.build_dir, "inference.pkl")
with open(dest, "wb") as f:
    pickle.dump((model, mcmc, posterior_samples), f)


In [ ]:
dest = os.path.join(model.build_dir, "numpyro_data.nc")
az.to_netcdf(numpyro_data, dest)


'/home/vishu/repos/hbmep-paper/reports/paper/tms/link-comparison/rectified_logistic/numpyro_data.nc'